In [6]:
# import torch
# from torch import nn
# import blobconverter
# from onnxsim import simplify
# import onnx
# import torch.nn.functional as F

# class Model(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.fc1 = nn.Linear(4, 100)
#         self.fc2 = nn.Linear(100, 100)
#         self.fc3 = nn.Linear(100, 3)
#         self.softmax = nn.Softmax(dim=1) 

#     def forward(self, X):
#         X = F.relu(self.fc1(X))
#         X = self.fc2(X)
#         X = self.fc3(X)
#         return X



# model = Model()
# shape = (10,4)
# X = torch.ones(shape, dtype=torch.float32)

# onnx_file = "../../results/test.onnx"
# torch.onnx.export(
#     model,
#     X,
#     onnx_file,
#     opset_version=12,
#     do_constant_folding=True,
# )


# onnx_simplified_path = "../../results/onnx_simple.onnx"
# onnx_model =  onnx.load(onnx_file)
# model_simp, check = simplify(onnx_model)
# onnx.save(model_simp, onnx_simplified_path)


# blobconverter.from_onnx(
#     model=onnx_file,
#     data_type="FP16",
#     shaves=6,
#     use_cache=False,
#     output_dir="../../results",
#     optimizer_params=[]
# )

In [7]:
import torch
from torch import nn
import blobconverter
from onnxsim import simplify
import onnx


import os
import torch
from torch import nn
from torchvision.models import resnet18,mobilenet_v2,ResNet18_Weights
import torchvision
import torchvision.transforms as transforms
from torchvision.models import MobileNetV2

data_dir = os.path.abspath("../../data")
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), transforms.RandomErasing()])
trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

dataiter = iter(trainloader) 
images, labels = next(dataiter)

model = resnet18(weights = ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(in_features=512,out_features=10)
model.load_state_dict(torch.load("../../results/Crossentropy_Resnet18_model.pth"))  


shape = (10,4)
X = torch.ones(shape, dtype=torch.float32)

onnx_file = "../../results/test.onnx"
torch.onnx.export(
    model,
    images,
    onnx_file,
    opset_version=12,
    do_constant_folding=True,
)


onnx_simplified_path = "../../results/onnx_simple.onnx"
onnx_model =  onnx.load(onnx_file)
model_simp, check = simplify(onnx_model)
onnx.save(model_simp, onnx_simplified_path)


blobconverter.from_onnx(
    model=onnx_file,
    data_type="FP16",
    shaves=8,
    use_cache=False,
    output_dir="../../results",
    optimizer_params=[]
)

Files already downloaded and verified
[==========================                        ]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

